In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape

#### Scale Features to Speed-up Convergence

In [3]:
X = housing.data
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)

print(X_mean.shape)
print(X_std.shape)

(8,)
(8,)


In [4]:
scaled_X = (X - X_mean) / X_std

In [5]:
scaled_X.mean(axis=0)

array([ 6.60969987e-17,  5.50808322e-18,  6.60969987e-17, -1.06030602e-16,
       -1.10161664e-17,  3.44255201e-18, -1.07958431e-15, -8.52651283e-15])

In [6]:
scaled_X.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_X]

#### Batch Gradient Descent
* auto-gradient
* manual update vs. optimizer

In [8]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name="mse")

# gradients = 2 / m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]

# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch %i, mse=%.5f"%(epoch, mse.eval()))
        sess.run(training_op)
    best_theta = theta.eval()
    
best_theta

epoch 0, mse=3.51183
epoch 100, mse=0.60709
epoch 200, mse=0.53202
epoch 300, mse=0.52812
epoch 400, mse=0.52734
epoch 500, mse=0.52682
epoch 600, mse=0.52640
epoch 700, mse=0.52606
epoch 800, mse=0.52577
epoch 900, mse=0.52553


array([[ 2.0685525 ],
       [ 0.8650663 ],
       [ 0.12563613],
       [-0.3320512 ],
       [ 0.36050886],
       [-0.00245932],
       [-0.04066427],
       [-0.81640565],
       [-0.7911649 ]], dtype=float32)